# Query Above Options Timeseries

This notebook demonstrates how to query the `api_v1_above_options_timeseries` endpoint,
which returns probabilities and prices for "above" markets (daily, sampled every 5 minutes).

The endpoint supports two formats:
- **long**: One row per timestamp+horizon+strike combination
- **wide**: One row per timestamp, with columns for sorted strikes by position

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv("../.env.local")

from polybridge import PolybridgeClient
from datetime import datetime, timedelta, timezone

import pandas as pd

In [ ]:
client = PolybridgeClient(api_key=os.getenv("POLYBRIDGE_API_KEY"))

## Long Format

Long format returns one row per timestamp+horizon+strike combination.
Columns: `timestamp_bucket`, `spot_price`, `spot_price_exchange`, `relative_horizon`, `strike_price`, `prob_above`

In [ ]:
# Query with long format
end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(hours=8)

result_long = client.fetch_above_options_timeseries(
    asset="BTC",
    start_ts=start_time.isoformat(),
    end_ts=end_time.isoformat(),
    format="long",  # Long format: one row per timestamp+horizon+strike
    horizon="daily"  # Optional, defaults to "daily"
)

# Convert to DataFrame
rows_long = result_long["rows"]
df_long = pd.DataFrame(rows_long)

In [ ]:
# Display structure and sample data
print(f"Long format: {len(df_long)} rows")
print(f"\nColumns: {list(df_long.columns)}")
print(f"\nData types:")
print(df_long.dtypes)
print(f"\nFirst few rows:")
df_long.head(20)

In [ ]:
# Check unique values
print(f"Unique timestamps: {df_long['timestamp_bucket'].nunique()}")
print(f"Unique relative_horizons: {sorted(df_long['relative_horizon'].unique())}")
print(f"\nUnique strikes per horizon:")
for horizon in sorted(df_long['relative_horizon'].unique()):
    strikes = df_long[df_long['relative_horizon'] == horizon]['strike_price'].nunique()
    print(f"  {horizon}: {strikes} unique strikes")

## Wide Format

Wide format returns one row per timestamp.
Columns include:
- `timestamp_bucket`, `spot_price`, `spot_price_exchange`
- For each relative_horizon (e.g., "next", "next+1", ...):
  - `{horizon}_strike_1` through `{horizon}_strike_11` (sorted by strike price, ascending)
  - `{horizon}_prob_above_1` through `{horizon}_prob_above_11`

Example column names: `next_strike_1`, `next_prob_above_1`, `next_plus_1_strike_1`, etc.

In [ ]:
# Query with wide format
result_wide = client.fetch_above_options_timeseries(
    asset="BTC",
    start_ts=start_time.isoformat(),
    end_ts=end_time.isoformat(),
    format="wide",  # Wide format: one row per timestamp
    horizon="daily"  # Optional, defaults to "daily"
)

# Convert to DataFrame
rows_wide = result_wide["rows"]
df_wide = pd.DataFrame(rows_wide)

In [ ]:
# Display structure and sample data
print(f"Wide format: {len(df_wide)} rows")
print(f"\nTotal columns: {len(df_wide.columns)}")
print(f"\nColumn names (first 20):")
print(list(df_wide.columns)[:20])
print(f"\nFirst few rows:")
df_wide.head()

In [ ]:
# Check how many strikes are present per horizon
strike_columns = [col for col in df_wide.columns if '_strike_' in col]
print(f"Total strike columns: {len(strike_columns)}")
print(f"\nSample strike columns:")
for col in sorted(strike_columns)[:20]:
    non_null = df_wide[col].notna().sum()
    print(f"  {col}: {non_null} non-null values")